<a href="https://colab.research.google.com/github/ML-Guy/DummyGenerartorData/blob/master/tf2_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [0]:
!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf

     |████████████████████████████████| 79.9MB 61.1MB/s 
     |████████████████████████████████| 61kB 22.1MB/s 
     |████████████████████████████████| 419kB 42.2MB/s 
     |████████████████████████████████| 3.0MB 33.9MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
import tensorflow_datasets as tfds

dataset = tfds.load('fashion_mnist',as_supervised=True)
ds_train, ds_test = dataset['train'],dataset['test']

In [0]:
mnist_train = tfds.load(name="mnist", split=tfds.Split.TRAIN)
# tfds.load is really a thin conveninence wrapper around DatasetBuilder. 
# We can accomplish the same as above directly with the MNIST DatasetBuilder
mnist_builder = tfds.builder("mnist")
mnist_builder.download_and_prepare()
mnist_train = mnist_builder.as_dataset(split=tfds.Split.TRAIN)

All DatasetBuilders expose various data subsets defined as tfds.Splits (typically tfds.Split.TRAIN and tfds.Split.TEST). A given dataset's splits are defined in tfds.DatasetBuilder.info.splits and are accessible through tfds.load and tfds.DatasetBuilder.as_dataset, both of which take split= as a keyword argument.

tfds enables you to further manipulate splits by combining them or subsplitting them up

In [0]:
combined_split = tfds.Split.TRAIN + tfds.Split.TEST
ds = tfds.load("mnist", split=combined_split)
# Ds will iterate over all splits merged together
ds = tfds.load("mnist", split=tfds.Split.ALL)

## Subsplit
You have 3 options for how to get a thinner slice of the data than the base splits, all based on tfds.Split.subsplit.

In [0]:
train_half_1, train_half_2 = tfds.Split.TRAIN.subsplit(2) #
first_10_percent = tfds.Split.TRAIN.subsplit(tfds.percent[:10])
last_2_percent = tfds.Split.TRAIN.subsplit(tfds.percent[-2:])
middle_50_percent = tfds.Split.TRAIN.subsplit(tfds.percent[25:75])
half, quarter1, quarter2 = tfds.Split.TRAIN.subsplit([2, 1, 1])
# composition : Split the combined TRAIN and TEST splits into 2
first_half, second_half = (tfds.Split.TRAIN + tfds.Split.TEST).subsplit(2)

#Invalid Operations: a split cannot be added twice, and subsplitting can only happen once.
split = tfds.Split.TRAIN.subsplit(tfds.percent[:25]) + tfds.Split.TRAIN     # TRAIN included twice
split = (tfds.Split.TRAIN.subsplit(tfds.percent[:25]) + tfds.Split.TEST).subsplit(tfds.percent[0:50])   # Subsplit of subsplit

dataset = tfds.load("mnist", split=train_half_1)


List of datasets : https://www.tensorflow.org/datasets/datasets

image_label_folder : Generic image classification dataset created from manual directory.
  Reference : https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image/image_folder.py
  
  The data directory should have the following structure:
  ```
  path/to/manual_dir/<dataset_name>/
    split_name/  # Ex: 'train'
      label1/  # Ex: 'airplane' or '0015'
        xxx.png
        xxy.png
        xxz.png
      label2/
        xxx.png
        xxy.png
        xxz.png
    split_name/  # Ex: 'test'
      ...
  ```
  To use it:
  ```
  builder = tfds.image.ImageLabelFolder('<dataset_name>')
  dl_config = tfds.download.DownloadConfig(manual_dir='path/to/manual_dir/')
  builder.download_and_prepare(download_config=dl_config)
  print(builder.info)  # Splits, num examples,... automatically extracted
  ds = builder.as_dataset(split='split_name')
  ```
  Or with load:
  ```
  dl_config = tfds.download.DownloadConfig(manual_dir='path/to/manual_dir/')
  tfds.load(
      'image_label_folder',
      split='split_name'
      builder_kwargs=dict(dataset_name='<dataset_name>'),
      download_and_prepare_kwargs=dict(download_config=dl_config),
  )
  ```

![DatasetBuilder abstraction layers](https://www.tensorflow.org/datasets/dataset_layers.png)

In [0]:
# Create custom dataset
class DatasetBuilder:
  # Source data -> Preprocessed files
  def download_and_prepare:
    
  # Preprocessed files -> tf.data.Dataset 
  def as_dataset:
    
  # Dataset metadata: features, stats, etc.
  def info:

https://www.youtube.com/watch?v=oFFbKogYdfc&t=349s

Example: https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image/celeba.py

Info is good. Three step process is also good. still complex to build new dataset. 
Feature column, tf.transform and tf.data.Datasets all are have overlapping functionlities/methods to reuse pipeline b/w train and inference runs. Beam pipeline based methods would have been great.



In [0]:
#ds = create_dataset()

def scale(image,label):
  image = tf.cast(image,tf.float32)
  image /= 255
  return image,label

ds_train = ds_train.map(preprocess,num_parallel_calls=10).map(scale)
ds_train = ds_train.shuffle(1024).batch(64).repeat(10).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(scale).batch(64)

In [0]:
model=MyModel()

Keras's way :Simple and succient

In [0]:
model.compile(optmizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(xtrain,ytrain,epoch=5)

With Gradient Tape: More customisable 
con: increase in tech debt - more difficult to understand other's code

In [0]:
with tf.GradientTape() as tape:
  logits = model(images)
  loss_value = loss(logits,labels)
  
grads = tape.gradient(loss_value,model.trainable_variables)
optimizer.apply_gradients(zip(grads,model.trainable_variables))

In [0]:
@tf.function
def f(x):
  return x

print(tf.autograph.to_code(f)) #To see how tf.function changes py func to graph.


# Pending List

- https://www.tensorflow.org/model_optimization 
- Tf probability 
- https://www.tensorflow.org/lite/guide/get_started#2_convert_the_model_format 
- TF Data Validation,  TF transform, TF Modal Analysis, TF serving, WhatIf, Tensorboard, hub 
- [ Good practice python guide (google)](https://github.com/google/styleguide/blob/gh-pages/pyguide.md) and [PEP 8 style guide](https://www.python.org/dev/peps/pep-0008/)
- [Apache Beam Programming Guide](https://beam.apache.org/documentation/programming-guide/)
- 